In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF
# TODO START

**Imputer**

* Для замены `пропусков` в признаках `feat_num` и `feat_cat` используем разные стратегии:
    * `feat_num` - `strategy="mean"`
    * `feat_cat` - `strategy="most_frequent"`
        * 
* strategy="mean" — заполнять пропуски средним по колонке

* strategy="median" — заполнять медианой

* strategy="most_frequent" — заполнять самым частым значением (мода)

* strategy="constant" — заполнять константой (fill_value=...)

In [4]:
# Замена пропусков с помощью SimpleImputer
df = pd.DataFrame({
    "A": [pd.NA, "F", "W", "F", "W"],   # категориальный
    "B": [20, None, 30, 40, 50],         # числовой
    "C": [300, 200, 300, None, pd.NA]    # числовой (с pd.NA)
})

# 1) Заменяем pd.NA -> np.nan
df = df.replace({pd.NA: np.nan})

print('Заменяем pd.NA на np.nan : \n', df)

# TODO Определяем типы колонок
# 2) Импутация: mean для числовых, most_frequent для категориальных
cat_cols = ["A"]
num_cols = ["B", "C"]

# TODO 1-й вариант
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", SimpleImputer(strategy="mean"), num_cols),
#         ("cat", SimpleImputer(strategy="most_frequent"), cat_cols)
#     ]
# )


# TODO 2-й вариант
# Импутеры
cat_imputer = SimpleImputer(strategy="most_frequent")
num_imputer = SimpleImputer(strategy="mean")

# Трансформер
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_imputer, num_cols),
        ("cat", cat_imputer, cat_cols)
    ]
)


# TODO обратно собираем в DF cat и num признаки
result = preprocessor.fit_transform(df)
df_filled = pd.DataFrame(result, columns=num_cols + cat_cols)

print(result)



Заменяем pd.NA на np.nan : 
      A  B  C
0    a  1  4
1    a  2  5
2  NaN  3  6
[[1.0 4.0 'a']
 [2.0 5.0 'a']
 [3.0 6.0 'a']]



**ColumnTransformer ( SimpleImputer() и get_dummies()) - вместе не работают**

**Правила**

* в ColumnTransformer обязательно:
    * все скобки ColumnTransformer `( [ ( имя, трансформатор, столбцы ) ] )`
    * три обязательных компонента `имя, трансформатор, столбцы` должны быть `одним объектом`

In [6]:
df = pd.DataFrame({
    "A": [pd.NA, "F", "W", "F", "W"],   # категориальный
    "B": [20, None, 30, 40, 50],         # числовой
    "C": [300, 200, 300, None, pd.NA]    # числовой (с pd.NA)
})

print('Исходный DF : \n', df, '\n')

# TODO Заменяем pd.NA -> np.nan
data = df.replace({pd.NA: np.nan})

name_col_odj = data.select_dtypes(include=['object', 'category']).columns
name_col_int = data.select_dtypes(include=['int', 'float']).columns

# TODO Трансформаторы для catfeat и numfeat
ohe_obj = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
imput_obj = SimpleImputer(strategy='most_frequent')

scaler_int =StandardScaler()
imput_int = SimpleImputer(strategy='mean')

# TODO pipeline для признаков catfeat и numfeat
pipe_obj = make_pipeline(imput_obj,ohe_obj)
pipe_int = make_pipeline(imput_int, scaler_int)


# TODO SimpleImputer и get_dummies Не Работает
# pipe_obj = make_pipeline(SimpleImputer(strategy='most_frequent'), pd.get_dummies(data=data['A']).astype(int))



data_trsf_obj = ColumnTransformer([('object', pipe_obj, name_col_odj)])
data_trsf_int = ColumnTransformer([('numeric', pipe_int, name_col_int)])


# TODO Имена новых признаков catfeat после ОНЕ и numfeat берём из ColumnTransformer
data_obj = data_trsf_obj.fit_transform(data)
print('Имена новых признаков ОНЕ : \n', data_trsf_obj.get_feature_names_out(), '\n')

data_int = data_trsf_int.fit_transform(data)
print('Имена Стандартизированных признаков int : \n', data_trsf_int.get_feature_names_out(), '\n')

df_obj = pd.DataFrame(data_obj, columns=[data_trsf_obj.get_feature_names_out()])
df_int = pd.DataFrame(data_int, columns=[data_trsf_int.get_feature_names_out()])

data_df = pd.concat([df_int, df_obj], axis=1)

print('Преобразованный DF : ')
data_df

Исходный DF : 
       A  B  C
0     a  1  4
1     a  2  5
2  <NA>  3  6 

Имена новых признаков ОНЕ : 
 ['object__A_a'] 

Имена Стандартизированных признаков int : 
 ['numeric__B' 'numeric__C'] 

Преобразованный DF : 


,numeric__B,numeric__C,object__A_a
0,-1.224745,-1.224745,1.0
1,0.000000,0.000000,1.0
2,1.224745,1.224745,1.0


**pipline DF -> Стандартизация  -> LogReg-модель**

* `C=10` - `обратная сила` регуляризации:
    * Чем `меньше C`, тем `сильнее` регуляризация `(больше штраф)`.
    * Чем `больше C`, тем `меньше` регуляризация.


In [154]:

X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1 , random_state=42)

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# C = 1 / λ (обратная величина коэффициента регуляризации).
pipe_lg = make_pipeline(StandardScaler(), LogisticRegression(C=10))

pipe_lg.fit(X_train, y_train)
y_pred = pipe_lg.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall: ", recall_score(y_test, y_pred, average="macro"))
print("F1-score: ", f1_score(y_test, y_pred, average="macro"), '\n')

(1000, 4) (1000,)
Accuracy:  0.9
Precision:  0.9001600640256102
Recall:  0.9
F1-score:  0.8999899989999 



# 1

**remainder**
   * `remainder='passthrough'` - оставшиеся столбцы сохранить
   * `remainder='drop'` - оставшиеся столбцы удалить
                       

In [162]:

X = pd.DataFrame(
    {'city': ['London', 'London', 'Paris', 'Sallisaw'],
     'title': ["His Last Bow", "How Watson Learned the Trick",
               "A Moveable Feast", "The Grapes of Wrath"],
     'expert_rating': [500, 300, 400, 500],
     'user_rating': [0.4, 0.5, 0.4, 0.3]})

column_trans = ColumnTransformer(
    [('city_category', OneHotEncoder(dtype='int'),['city']),
     ('title_bow', CountVectorizer(), 'title')],
    remainder='passthrough')

column_trans.fit_transform(X)

array([[1.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00,
        1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        5.e+02, 4.e-01],
       [1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+00,
        0.e+00, 1.e+00, 0.e+00, 0.e+00, 1.e+00, 1.e+00, 1.e+00, 0.e+00,
        3.e+02, 5.e-01],
       [0.e+00, 1.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00,
        4.e+02, 4.e-01],
       [0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00, 1.e+00, 0.e+00, 0.e+00,
        0.e+00, 0.e+00, 0.e+00, 1.e+00, 1.e+00, 0.e+00, 0.e+00, 1.e+00,
        5.e+02, 3.e-01]])

# 2

* `remainder=MinMaxScaler()` - оставшиеся столбцы стандартизировать  

In [165]:

# Данные
df = pd.DataFrame({
    'age': [20, 30, 40],
    'salary': [2000, 3000, 4000],
    'city': ['Moscow', 'SPb', 'Kazan']
})

# Преобразования
ct = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age']),       # только age стандартизируем
        ('cat', OneHotEncoder(), ['city'])        # категориальные кодируем
    ],
    remainder=MinMaxScaler()  # оставшийся столбец (salary) нормализуем
)

result = ct.fit_transform(df)
print(result)


[[-1.22474487  0.          1.          0.          0.        ]
 [ 0.          0.          0.          1.          0.5       ]
 [ 1.22474487  1.          0.          0.          1.        ]]


# Вариант 1
# CountVectorizer → `создаёт словарь и считает частоты.`

# TfidfTransformer → `пересчитывает эти частоты в веса, нормализация и штраф`

**CountVectorizer + TfidfTransformer**

*   Берём коллекцию текстов (например, `список строк`).

*   Строим `словарь` всех `уникальных` слов.

*   Для каждого `текста`(`строки`) считаем, сколько раз каждое слово встречается.

*   В результате получаем разреженную матрицу размера (`число документов(строк) `× `число слов(токенов)`).

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

docs = [
    "кот любит молоко",
    "собака любит кость",
    "кот и собака друзья"
]

# docs = [
#     
#         "Human machine interface for lab abc computer applications",
#         "A survey of user opinion of computer system response time",
#         "The EPS user interface management system",
#         "System and human system engineering testing of EPS",
#         "Relation of user perceived response time to error measurement",
#         "The generation of random binary unordered trees",
#         "The intersection graph of paths in trees",
#         "Graph minors IV Widths of trees and well quasi ordering",
#         "Graph minors A survey",
#     
# ]


count_vect = CountVectorizer()
X_vect = count_vect.fit_transform(docs)

print(f"Размер матрицы: {X_vect.shape}  # {X_vect.shape[0]} документов × {X_vect.shape[1]} уникальных слов: \n", X_vect.shape, '\n')
print(X_vect.toarray())

tfidf_transform = TfidfTransformer()

X_word = tfidf_transform.fit_transform(X_vect)

print('X_word - CountVectorizer + TfidfTransformer, разрежённая матрица (sparse matrix) : \n', X_word, '\n')


Размер матрицы: (3, 6)  # 3 документов × 6 уникальных слов: 
 (3, 6) 

[[0 0 1 1 1 0]
 [0 1 0 1 0 1]
 [1 0 1 0 0 1]]
X_word - CountVectorizer + TfidfTransformer, разрежённая матрица (sparse matrix) : 
   (0, 2)	0.5178561161676974
  (0, 3)	0.5178561161676974
  (0, 4)	0.680918560398684
  (1, 1)	0.680918560398684
  (1, 3)	0.5178561161676974
  (1, 5)	0.5178561161676974
  (2, 0)	0.680918560398684
  (2, 2)	0.5178561161676974
  (2, 5)	0.5178561161676974 



# Вариант 2

# TfidfVectorizer  `→ «всё в одном»`

**Дублируемость классов - Векторизация текста**

(1, 9)    0.3516654787271834

* 1 — индекс документа (строка, начиная с 0),

* 9 — индекс слова в словаре (колонка),

* 0.3516... — вес TF-IDF для этого слова в этом документе.

Значения лежат в диапазоне от 0 до 1:

1 → `тексты идентичны`,

0 → `тексты вообще не похожи.`

In [8]:
# import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# Допустим, у нас есть классы тикетов (категории обращений)


classes = [
    "кот любит молоко",
    "собака любит кость",
    "кот и собака друзья"
]

# classes = [
#     "Ошибка входа",
#     "Ошибка при входе в систему",
#     "Не могу войти в аккаунт",
#     "Проблема с оплатой",
#     "Ошибка при оплате",
#     "Сброс пароля",
#     "Проблема пароля",
#     "Ошибка восстановления пароля"
# ]

# classes = [
#     
#         "Human machine interface for lab abc computer applications",
#         "A survey of user opinion of computer system response time",
#         "The EPS user interface management system",
#         "System and human system engineering testing of EPS",
#         "Relation of user perceived response time to error measurement",
#         "The generation of random binary unordered trees",
#         "The intersection graph of paths in trees",
#         "Graph minors IV Widths of trees and well quasi ordering",
#         "Graph minors A survey",
#     
# ]


df = pd.DataFrame({'class_name': classes})

print('Исходный df : \n', df, '\n')


# 1️⃣ Преобразуем тексты в векторное представление (TF-IDF)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['class_name'])

print('X - vectorizer, sparse matrix >>> веса слов по документам : \n', X, '\n')

feature_name = vectorizer.get_feature_names_out(X)

print('get_feature_names_out -> Уникальные Слова : \n', feature_name)

print('Словарь tf-idf : \n', vectorizer.vocabulary_, '\n')


# TODO косинусные сходства - После Векторизации
# 2️⃣ Считаем попарные косинусные сходства
similarity = np.round(cosine_similarity(X), 4)

print('Матрица numpy косинусные сходства документов: \n', similarity, '\n')

# 3️⃣ Переведём в DataFrame для удобства
sim_df = pd.DataFrame(similarity, index=classes, columns=classes)

print('косинусные сходства : \n', sim_df, '\n')

# TODO cos Подобие документов через python цикл
# 4️⃣ Найдём пары классов, похожих сильнее порога
threshold = 0.3
similar_pairs = []

for i in range(len(sim_df)):  # Строки   
    for j in range(i+1, len(sim_df)):  # Столбцы, 👉 индекс по столбцам i+1, обходим диагональ со значением 1 
        if sim_df.iloc[i, j] > threshold:
            similar_pairs.append((
                df['class_name'][i],
                df['class_name'][j],
                round(sim_df.iloc[i, j], 3)
            ))

df = pd.DataFrame(similar_pairs, columns=['class_1', 'class_2', 'similarity'])
print('Дублируемые классы :\n', df)


Исходный df : 
             class_name
0     кот любит молоко
1   собака любит кость
2  кот и собака друзья 

X - vectorizer, sparse matrix >>> веса слов по документам : 
   (0, 2)	0.5178561161676974
  (0, 3)	0.5178561161676974
  (0, 4)	0.680918560398684
  (1, 3)	0.5178561161676974
  (1, 5)	0.5178561161676974
  (1, 1)	0.680918560398684
  (2, 2)	0.5178561161676974
  (2, 5)	0.5178561161676974
  (2, 0)	0.680918560398684 

get_feature_names_out -> Уникальные Слова : 
 ['друзья' 'кость' 'кот' 'любит' 'молоко' 'собака']
Словарь tf-idf : 
 {'кот': 2, 'любит': 3, 'молоко': 4, 'собака': 5, 'кость': 1, 'друзья': 0} 

Матрица numpy косинусные сходства документов: 
 [[1.     0.2682 0.2682]
 [0.2682 1.     0.2682]
 [0.2682 0.2682 1.    ]] 

косинусные сходства : 
                      кот любит молоко  собака любит кость  кот и собака друзья
кот любит молоко               1.0000              0.2682               0.2682
собака любит кость             0.2682              1.0000               0.2682
к

# Подобие документов через numpy - triu

**Ключевое - `удаляем дубликаты через обрезание симметричной матрицы подобия`**

**Результат всегда 2 столбца - `Потому что сравниваем 2 документа`**

**Параметр k в triu**

| k     | Что происходит        |
| ----- | --------------------- |
| `k=0` | включается диагональ  |
| `k=1` | диагональ исключается |
| `k=2` | ещё выше              |


In [9]:
# TODO k=1 в np.triu обнуляет главную диагональ

threshold = 0.3

print('Исходный массив подобия документов : \n', similarity, '\n')

#  Условие фильтра значения порога
cond = np.triu(similarity, k=1) > threshold

#  Фильтруем координаты значений по условию
x, y = np.where(cond)  
print('координаты значений по условию "> порога" : \n', x, y, '\n')

#  Обрезка нижней части массива подобия + to df
df = pd.DataFrame(np.triu(similarity, k=1), columns=classes, index=classes)

# i - индекс строк, j - индекс столбов
df_sim = pd.DataFrame([(df.index[i], df.columns[j], df.values[i, j]) for i, j in zip(x, y)], 
                      columns=['doc_similarity_1', 'doc_similarity_2', 'similarity'])

df_sim

Исходный массив подобия документов : 
 [[1.     0.2682 0.2682]
 [0.2682 1.     0.2682]
 [0.2682 0.2682 1.    ]] 

координаты значений по условию "> порога" : 
 [] [] 



,doc_similarity_1,doc_similarity_2,similarity


# NMF
** NMF - `X-матрца -> documents × topics -> topics × words`**

** LDA - `word × topics -> topics × document` **


In [20]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=2, random_state=42)
W = nmf.fit_transform(X)   # documents × topics
H = nmf.components_        # topics × words

print(f'{W.shape[0]} документа × {W.shape[1]}  темы : \n', W, '\n')

print(f'{H.shape[0]} Тем(ы) × {H.shape[1]} слов : \n', H )


3 документа × 2  темы : 
 [[5.10343691e-01 1.05536020e-04]
 [5.10487257e-01 0.00000000e+00]
 [0.00000000e+00 8.82373199e-01]] 

2 Тем(ы) × 6 слов : 
 [[0.00000000e+00 6.67117617e-01 5.07156785e-01 1.01457760e+00
  6.66929994e-01 5.07299482e-01]
 [7.71689974e-01 0.00000000e+00 5.86925227e-01 9.87338634e-09
  4.61618718e-05 5.86855023e-01]]


3

In [12]:
W
# H

array([[5.10343691e-01, 1.05536020e-04],
       [5.10487257e-01, 0.00000000e+00],
       [0.00000000e+00, 8.82373199e-01]])